<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/first%20some%20ques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Simulation Parameters
T_values = [0.5, 2.0]  # Temperatures
rho_values = [0.5, 0.7]  # Densities (N/V)
N = 100  # Number of Lennard-Jones spheres
epsilon = 1  # Lennard-Jones well depth
sigma = 1  # Lennard-Jones parameter
k_B = 1  # Boltzmann constant (dimensionless units)
n_steps = 50000  # Number of Monte Carlo steps
max_disp = 0.1  # Maximum displacement step
num_samples = 20  # Number of independent samples

# Store results
results = []
performance_data = []

for T in T_values:
    beta = 1 / (k_B * T)
    for rho in rho_values:
        L = (N / rho) ** (1 / 3)  # Box length
        np.random.seed(42)
        positions = L * np.random.rand(N, 3)

        E_samples = []
        acceptance = 0

        start_time = time.time()

        for step in range(n_steps):
            i = np.random.randint(N)
            old_pos = positions[i].copy()
            new_pos = old_pos + max_disp * (2 * np.random.rand(3) - 1)
            new_pos = np.mod(new_pos, L)  # Apply periodic boundary conditions

            dE = 0
            for j in range(N):
                if j != i:
                    r_old = np.linalg.norm(old_pos - positions[j])
                    r_new = np.linalg.norm(new_pos - positions[j])

                    if r_old < L / 2:
                        E_old = 4 * epsilon * ((sigma / r_old) ** 12 - (sigma / r_old) ** 6)
                    else:
                        E_old = 0

                    if r_new < L / 2:
                        E_new = 4 * epsilon * ((sigma / r_new) ** 12 - (sigma / r_new) ** 6)
                    else:
                        E_new = 0

                    dE += (E_new - E_old)

            if dE < 0 or np.random.rand() < np.exp(-beta * dE):
                positions[i] = new_pos
                acceptance += 1

            if step % (n_steps // num_samples) == 0 and step > n_steps // 2:
                total_energy = sum(
                    4 * epsilon * ((sigma / np.linalg.norm(positions[i] - positions[j])) ** 12 -
                                   (sigma / np.linalg.norm(positions[i] - positions[j])) ** 6)
                    for i in range(N) for j in range(i))
                E_samples.append(total_energy / N)

        end_time = time.time()
        steps_per_second = n_steps / (end_time - start_time)
        performance_data.append((T, rho, steps_per_second))

        avg_E = np.mean(E_samples)
        std_E = np.std(E_samples)
        results.append((T, rho, avg_E, std_E, len(E_samples)))

# Display Results
print("Simulation Results:")
for T, rho, avg_E, std_E, num_samples in results:
    print(f'T={T}, Density={rho}: <E>/N = {avg_E:.3f}, Std Dev = {std_E:.3f}, Samples = {num_samples}')

print("\nPerformance Measurements:")
for T, rho, steps_per_second in performance_data:
    print(f'T={T}, Density={rho}: {steps_per_second:.2f} steps/second')

# Summary Takeaways:
# 1. Higher density simulations take longer due to increased particle interactions.
# 2. Lower temperatures exhibit slower equilibration due to fewer accepted moves in the Metropolis algorithm.
plt.plot(s.energies[0:]/N)
plt.show()


Simulation Results:
T=0.5, Density=0.5: <E>/N = -3.656, Std Dev = 0.115, Samples = 9
T=0.5, Density=0.7: <E>/N = -4.097, Std Dev = 0.072, Samples = 9
T=2.0, Density=0.5: <E>/N = -1.783, Std Dev = 0.096, Samples = 9
T=2.0, Density=0.7: <E>/N = -2.468, Std Dev = 0.119, Samples = 9

Performance Measurements:
T=0.5, Density=0.5: 971.16 steps/second
T=0.5, Density=0.7: 926.07 steps/second
T=2.0, Density=0.5: 1029.03 steps/second
T=2.0, Density=0.7: 1062.28 steps/second
